In [ ]:
# ASSNP-16.py
# 2021-11-29
#
from datetime import datetime
dt_start = datetime.now()
#
import os
# to fix matplotlib warning
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
path = os.getcwd()
print('當前工作目錄 ==>', path)
#
import numpy as np
import matplotlib as mpl
mpl.rcParams.update({'figure.max_open_warning': 0})
import matplotlib.pyplot as plt
import matplotlib.font_manager as fontManager
#
from shapely.geometry import LineString
#
def set_graphic_area(width,height) :
        
    cm2inch = 1/2.54    # inch per cm
    #
    # define graphic area
    
    #
    left_margin = 2.0   # cm
    right_margin = 2.0  # cm
    #
    figure_width  = width  # cm , from xmin to xmax
    figure_height = height # cm , from ymin to ymax
    #
    top_margin = 2.0    # cm
    bottom_margin = 2.0 # cm
    #
    box_width = left_margin + figure_width + right_margin   # cm
    box_height = top_margin + figure_height + bottom_margin # cm
    #
    top_value    = 1.0 - top_margin / box_height
    bottom_value = bottom_margin / box_height
    left_value   = left_margin / box_width
    right_value  = 1.0 - right_margin / box_width
    #
    return (box_width*cm2inch,box_height*cm2inch,top_value,bottom_value,left_value,right_value,width)
#
# end of def
#
def new_plot_setting() :
    tup7 = set_graphic_area(45,70)
    fig = plt.figure(figsize=(tup7[0], tup7[1]))
    ax = fig.add_subplot(1,1,1)
    fig.subplots_adjust(
                top    = tup7[2] ,
                bottom = tup7[3] ,
                left   = tup7[4] ,
                right  = tup7[5] ,
                )
    #
    plt.xlim(-5, 40)
    plt.ylim(20, 90)    
    #
    x_ticks = []
    y_ticks = []
    #
    plt.xticks(x_ticks, fontsize=24)
    plt.yticks(y_ticks, fontsize=24)
    #
    plt.grid(False)
    ax.set_axis_on()
    plt.gca().set_aspect('equal', adjustable='box') # set X,Y same ratio & scale
# end def
#
path_data    = './data/'
path_graphic = './graphic/'
#
mpl.rcParams['pdf.fonttype'] = 42
myfont = fontManager.FontProperties(fname='./fonts/msjh.ttc')
#
# -------------------------------------------------------------------------------
#
tup7 = set_graphic_area(45,70)
fig = plt.figure(figsize=(tup7[0], tup7[1]))
ax = fig.add_subplot(1,1,1)
fig.subplots_adjust(
                top    = tup7[2] ,
                bottom = tup7[3] ,
                left   = tup7[4] ,
                right  = tup7[5] ,
                )
#
plt.xlabel('X座標', fontproperties=myfont, fontsize=26)
plt.ylabel('Y座標', fontproperties=myfont, fontsize=26)
#
plt.xlim(-5, 40)    # xlim, ylim, need to be same
plt.ylim(20, 90)    
#
x_ticks = []
y_ticks = []
#
plt.xticks(x_ticks, fontsize=24)
plt.yticks(y_ticks, fontsize=24)
#
plt.grid(False)
ax.set_axis_on()
plt.gca().set_aspect('equal', adjustable='box') # set X,Y same ratio & scale
#
# --------------------------------------------------------------------------
#
# 逐筆讀入資料 dem-all-7.csv
# Read from the file into the array data(:,:)
# array data shape is (...., 3)
#
# Columns:
# data[..., 0] is array of float number
# data[..., 1] is array of float number
# data[..., 2] is array of float number
#
# 台灣橫麥卡托二度分帶投影座標系統（TM2）
#
float_To_int = np.vectorize(np.int32)
#
data2 = np.loadtxt(path_data + 'dem-all-7.csv',delimiter=',',dtype='float')
min_item = np.amin(data2, axis=0) # axis=0 --> min of each column
max_item = np.amax(data2, axis=0) # axis=0 --> max of each column
print ('Left Down point:', float_To_int(min_item[0]), float_To_int(min_item[1]))
print ('Right Up point: ' , float_To_int(max_item[0]), float_To_int(max_item[1]))
print ('height (min, max)', min_item[2], max_item[2])
print ('\n')
#
# Left Down point:    0, 2040
# Right  Up point: 3800, 9000
# height (min, max) 0.0, 357.21
#
TM2_X = float_To_int(data2[..., 0]) # 二度分帶 X座標
TM2_Y = float_To_int(data2[..., 1]) # 二度分帶 Y座標
TM2_Z = data2[..., 2]               # DEM file z value, float type
#
# XYlist is sorted (x, y), first order is y, then x
#
XYlist = list(zip(TM2_X,TM2_Y))
#
# X, Y value
X = np.arange(-100, 3820, 20)
Y = np.arange(2040, 9020, 20)
Xmesh, Ymesh = np.meshgrid(X, Y)  # x-y 平面的網格
#
# X -100 .. 3800
# Y 2040 .. 9000
# 6980/20 ==> 349
# 3920/20 ==> 196
#
Zmesh = np.zeros(shape=(349,196),dtype=np.float16)
i = 0 ; j = 0
#
for y in np.arange(2040, 9020, 20) :
    print('y is', y)
    z = np.zeros(shape=(196),dtype=np.float16)
    zi = 0
    for x in np.arange(-100, 3820, 20):
        if (x,y) in XYlist :
            z[zi] = TM2_Z[i]
            i = i + 1
        else :
            z[zi] = 0.0
        # end if
        zi = zi + 1
    #end for
    #
    Zmesh[j] = z
    j = j + 1
#end for
print ('\n')
#
factor_3D = 0.01
Xmesh_3D = factor_3D * Xmesh
Ymesh_3D = factor_3D * Ymesh
Zmesh_3C = np.around(Zmesh)
Zmesh_3D = Zmesh_3C.astype(int)
#
# levels
# Determines the number and positions of the contour lines
# If an int n which tries to automatically choose no more than n+1 "nice" contour levels between vmin and vmax. (int n ---> 15 or 18)
# If array-like, draw contour lines at the specified levels. The values must be in increasing order.
#
color_map = [( 54/255, 33/255, 95/255), ( 68/255, 81/255,191/255),
             ( 69/255,140/255,253/255), ( 34/255,197/255,226/255),
             ( 37/255,236/255,167/255), (117/255,254/255, 92/255),
             (188/255,245/255, 52/255), (238/255,207/255, 58/255),
             (254/255,153/255, 44/255), (197/255, 38/255,  3/255),
             (146/255, 11/255,  1/255), (237/255, 85/255, 16/255),
             (178/255,  0/255,255/255)]
#
levels = (15,30,60,90,120,150,180,210,240,270,300,330,350)
i = 0
for level in levels : 
    if len(str(level)) == 2 :
        str_level = '0' + str(level)
    else :
        str_level = str(level)
    # end if
    contour_set = plt.contour(Xmesh_3D, Ymesh_3D, Zmesh_3D, [level], colors=color_map) 
    new_plot_setting()
    #
    # return paired points making up each line segment
    path_set = contour_set.collections[0].get_paths()
    # To get a NumPy array of the coordinates, use the Path.vertices attribute.
    print('dem level', level)
    print("path_set total length is", len(path_set))
    #
    j = 1
    for path in path_set :
        # grab the path coordinates
        path_coords = path.vertices
        print('path', j)
        xy_list = []
        for xy in path_coords :
            # xy_list.append(tuple(xy)) # <--- more precise
            xy_list.append(tuple([round(xy[0],3),round(xy[1],3)]))
        # end for
        # print('====================================')
        #
        if len(xy_list) > 1 :
            path_line = LineString(xy_list)
            xs, ys = path_line.xy
            plt.fill(xs, ys, color=color_map[i])
        # end if
        j = j + 1
    # end for
    #
    plt.savefig(path_graphic + "ASSNP-16-dem-" + str_level + ".png", format="png",dpi=150, bbox_inches='tight', transparent=True)
    #
    i = i + 1
    if i < len(levels) :
        new_plot_setting()
    # end if
# end for
#
print(dt_start)
dt_end = datetime.now()
print(dt_end)
#
print('done')
#